In [20]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from networkx import bipartite
import numpy as np
data = pd.read_csv('graph.csv')

In [21]:
data.head()

,person_id,item_number,Size
0,70718,30439-00,1
1,42079,K27.9,1
2,200106867,I49.2,1
3,200000976,C79.3,2
4,200098428,E87.1,1


In [22]:
edges = [tuple(x) for x in data[['person_id','item_number','Size']].values.tolist()]
B = nx.Graph()
B.add_nodes_from(data['person_id'].unique(), bipartite=0, label='person')
B.add_nodes_from(data['item_number'].unique(), bipartite=1, label='icd')
for row in edges:
    B.add_edge(row[0], row[1], rating=row[2])

# Now instead of spring_layout, use bipartite_layout

# First specify the nodes you want on left or top
left_or_top = data['person_id'].unique()

# Then create a bipartite layout
pos = nx.bipartite_layout(B, left_or_top)

# Pass that layout to nx.draw
#nx.draw(B,pos,node_color='#A0CBE2',edge_color='#00bb5e',width=1,
 #    edge_cmap=plt.cm.Blues,with_labels=True)

In [23]:
X = set(sub_data['person_id'])
len(X)

2056

In [24]:
P = bipartite.weighted_projected_graph(B,X)

In [25]:
df = nx.to_pandas_edgelist(P)

In [26]:
df

,source,target,weight
0,90112,82059,2
1,90112,148364,1
2,90112,99475,1
3,90112,157344,2
4,90112,82081,2
...,...,...,...
85407,200023996,122878,1
85408,98250,81898,1
85409,98250,122878,1
85410,8144,155635,1


In [27]:
df.to_csv("patient.graph", sep="\t", index = False, header = False)

In [29]:
content = pd.read_csv('patient.csv')

In [33]:
#del content["patient_id"]

In [34]:
content.to_csv('patient.content', index = False, header = False, sep = "\t")

In [35]:
####

In [36]:
df = pd.DataFrame(dict(
    #DEGREE = dict(P.degree),
    DEGREE_CENTRALITY = nx.degree_centrality(P),
    EIGENVECTOR = nx.eigenvector_centrality(P, weight = "weight"),
    #KATZ = nx.katz_centrality_numpy(P),
    CLOSENESS_CENTRALITY = nx.closeness_centrality(P),
    BETWEENNESS_CENTRALITY = nx.betweenness_centrality(P, weight = "weight"),
    CLUSTCOEF = nx.clustering(P, weight = "weight"),
))

In [37]:
df.reset_index(level=0, inplace=True)

In [38]:
df.head()

,index,DEGREE_CENTRALITY,EIGENVECTOR,CLOSENESS_CENTRALITY,BETWEENNESS_CENTRALITY,CLUSTCOEF
0,90112,0.015572,0.000176,0.308646,4.166971e-05,0.201297
1,16386,0.083698,0.010578,0.441597,4.788251e-03,0.072377
2,40963,0.002433,0.000010,0.278261,9.484339e-07,0.170999
3,200015876,0.003893,0.000143,0.315827,2.803019e-05,0.101190
4,131075,0.006813,0.000300,0.318975,8.605539e-07,0.170475


In [40]:
df.columns = ['person_id', 'DEGREE_CENTRALITY','EIGENVECTOR', 'CLOSENESS_CENTRALITY','BETWEENNESS_CENTRALITY','CLUSTCOEF']
df.to_csv("nodefeature.csv", index=None)
df_patient = pd.merge(df, content, how='right', on='person_id')
df_patient = df_patient.fillna(0)
df_patient.to_csv("patientml.csv", index = None)

In [41]:
df_patient.head()

,person_id,DEGREE_CENTRALITY,EIGENVECTOR,CLOSENESS_CENTRALITY,BETWEENNESS_CENTRALITY,CLUSTCOEF,gender,age,smoker,T2d
0,200132255,0.000973,0.000005,0.263413,0.000000,0.166667,1,0.504348,1,0
1,7673,0.004380,0.000453,0.333830,0.000001,0.175970,1,0.573913,1,1
2,200066270,0.056448,0.007856,0.415296,0.000793,0.117475,0,0.695652,0,0
3,140445,0.008759,0.001373,0.365491,0.000038,0.152606,1,0.347826,0,0
4,115693,0.006813,0.000164,0.319032,0.002754,0.098977,0,0.391304,1,0
